In [1]:
import json, os, sys,re

In [2]:
all = json.load(open('/home/johnny/Code/DP-Air-Traffic-Comm-Transcription/data/atco/PROMPT/metadata_en_zurich_test.json'))
wer_prop = open('metadata_en_zurich_tmp.txt').readlines()
count = 0
for line in wer_prop:
    line = re.sub(r'\s+', ' ', line).strip()
    file_name = line.split()[0].replace('.xml', '.wav')
    wer_count = int(line.split()[1])
    for item in all:
        if file_name in item['audio']:
            # print(item['audio'])
            s = sum([cal['val'] for cal in item['prompt-data']['long_callsigns']])
            if (s != wer_count):
                print(f"Error: {file_name} {s} {wer_count}")
    # count += wer_count

print(count)

Error: ATCO2-ASRdataset-v1_final/DATA/LSZH_ZURICH_ApronN_121_85MHz_20210414_140149.wav 0 1
0


In [22]:
from datasets import load_from_disk
from transformers import WhisperTokenizer, WhisperProcessor, WhisperForConditionalGeneration
import evaluate
import torch
import numpy as np
metric = evaluate.load("wer")
from torch.utils.data import Dataset, DataLoader

class PrepareDatasetAsInput:
    
    def __init__(self, feature_extractor, tokenizer_en, tokenizer_fr):
        self.feature_extractor = feature_extractor
        self.tokenizer_en = tokenizer_en
        self.tokenizer_fr = tokenizer_fr
            
    def prepare_dataset(self, batch):
        # load and resample audio data from 48 to 16kHz
        audio = batch["audio"]

        # compute log-Mel input features from input audio array
        batch["input_features"] = self.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

        # encode target text to label ids **** CHANGED FROM **sentence** TO **transcription**
        # if french, than use french tokenizer, english otherwise
        tokenizer = self.tokenizer_en
        if "lang" in batch:
            if batch["lang"] == "fr":
                tokenizer = self.tokenizer_fr

        batch["labels_fullts"] = tokenizer(batch["full_ts"]).input_ids
        batch["labels_shortts"] = tokenizer(batch["short_ts"]).input_ids

        return batch
    
    def prepare_dataset_self_prompt(self,batch):
        # load and resample audio data from 48 to 16kHz
        audio = batch["audio"]

        # compute log-Mel input features from input audio array
        batch["input_features"] = self.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

        # encode target text to label ids **** CHANGED FROM **sentence** TO **transcription**
        # if french, than use french tokenizer, english otherwise
        tokenizer = self.tokenizer_en
        if "lang" in batch:
            if batch["lang"] == "fr":
                tokenizer = self.tokenizer_fr
        
        # make prompt from the lables
        batch['fullts_prompt_ids'] = self.tokenizer_en.get_prompt_ids(batch["full_ts"]).tolist() # YOU NEED TO ADD TOLIST() because array cant be combined with list in the next lines
        batch['shortts_prompt_ids'] = self.tokenizer_en.get_prompt_ids(batch["short_ts"]).tolist() # YOU NEED TO ADD TOLIST() because array cant be combined with list in the next lines
        
        batch["labels_fullts"] = tokenizer(batch["full_ts"]).input_ids # building labels ids with prompt and tokens together
        batch["labels_shortts"] = tokenizer(batch["short_ts"]).input_ids

        return batch
    
    def prepare_dataset_with_prompt(self,batch):        
        # load and resample audio data from 48 to 16kHz
        audio = batch["audio"]

        # compute log-Mel input features from input audio array
        features = self.feature_extractor(
            audio["array"], 
            sampling_rate=audio["sampling_rate"],
            truncation=False,
            padding='max_length',
            return_attention_mask=True
        )

        # add input features to batch
        batch["input_features"] = features.input_features[0]
        
        # add attention mask to batch, potentially for data augmentation
        batch['attention_mask'] = features.attention_mask[0]

        # STORE THE CALLSIGNS FOR EVALUATION
        if ("prompt-data" in batch):
            batch['long_callsigns'] = batch['prompt-data']['long_callsigns']
            batch['short_callsigns'] = batch['prompt-data']['short_callsigns']
        else :
            batch['long_callsigns'] = {}
            batch['short_callsigns'] = {}
            
        # encode target text to label ids **** CHANGED FROM **sentence** TO **transcription**
        # if french, than use french tokenizer, english otherwise
        tokenizer = self.tokenizer_en
        if "lang" in batch:
            if batch["lang"] == "fr":
                tokenizer = self.tokenizer_fr

        # encode prompts to prompt ids - we assume that the dataset has a column `"prompt"` that contains the prompt for each example
        prompt_ids = tokenizer.get_prompt_ids(batch['prompt_fullts_AG']).tolist() # YOU NEED TO ADD TOLIST() because array cant be combined with list in the next lines

        batch["labels"] = prompt_ids + tokenizer(batch['full_ts']).input_ids # building labels ids with prompt and tokens together
        batch['prompt_ids'] = prompt_ids # add the prompt ids to the batch
        return batch


In [2]:
class ComputeMetrics:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        
    def compute_metrics(self,pred_text, reference_text):
        pred_ids = pred_text
        label_ids = reference_text

        # replace -100 with the pad_token_id
        label_ids[label_ids == -100] = self.tokenizer.pad_token_id

        # we do not want to group tokens when computing the metrics
        pred_str = self.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
        label_str = self.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

        wer = 100 * metric.compute(predictions=pred_str, references=label_str)

        return {"wer": wer}

    def compute_metrics2(self,pred_str,label_str):

        wer = 100 * metric.compute(predictions=pred_str, references=label_str)

        return {"wer": wer}

In [3]:
dataset_path = './data/en_ruzyne_test_ds'
# model_def = './test3-tiny-prompt/checkpoint-980'
# model_def = './checkpoint-980'
model_def = 'openai/whisper-tiny'

In [18]:
torch.cuda.synchronize()
torch.cuda.empty_cache()

In [27]:
model = WhisperForConditionalGeneration.from_pretrained(model_def).cuda()
processor = WhisperProcessor.from_pretrained(model_def,language="English", task="transcribe")
prepare_dataset = PrepareDatasetAsInput(processor.feature_extractor, processor.tokenizer, processor.tokenizer) #TODO handle FR

In [10]:
a=processor.tokenizer.decode(torch.tensor([50259, 50359, 50363, 48444,   289,   383,  8886,  1485, 18183,  1176,
        12845,  7545,   198,    34,  8886,  1485, 18183,  1176, 12845,   744,
         1321,  3385, 15702,  1229,   716, 17877,   472,  1045,  1451, 26601,
         1732,  2309,  4018,   198,   198, 27583,  1451,  1732,  2309,  4018,
          383,  8886,  1485, 18183,  1176, 12845,   220,   520,  5457, 11822,
          368,   129,   230, 50257]))
print(a)
b=processor.tokenizer.decode(torch.tensor([50259, 50359, 50363, 48444,   289,   383,  8886,  1485, 18183,  1176,
         12845,  7545,   198,    34,  8886,  1485, 18183,  1176, 12845,   744,
          1321,  3385, 15702,  1229,   716, 17877,   472,  1045,  1451, 26601,
          1732,  2309,  4018,   198,   198, 27583,  1451,  1732,  2309,  4018,
           383,  8886,  1485, 18183,  1176, 12845,   220,   520,  5457, 11822,
           368,   129,   230, 50257]))
print(b)

<|en|><|transcribe|><|notimestamps|>Radar CSA One Delta Zulu established
CSA One Delta Zulu roger contact Ruzyne Tower one three four decimal five six zero

three four five six zero CSA One Delta Zulu  pekný deň<|endoftext|>
<|en|><|transcribe|><|notimestamps|>Radar CSA One Delta Zulu established
CSA One Delta Zulu roger contact Ruzyne Tower one three four decimal five six zero

three four five six zero CSA One Delta Zulu  pekný deň<|endoftext|>


In [ ]:
processor.tokenizer.decode(torch.tensor([50258, 50259, 50359, 50363]))

'<|startoftranscript|><|en|><|transcribe|><|notimestamps|>'

In [27]:
processor.feature_extractor.padding_value

0.0

In [19]:
labels_ids=[50259, 50359, 50363,  9000,   578, 25866,  9116,  1246, 14868,  9436,
        18939,  6244, 50257,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100]

attention_mask = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
print(np.where(np.array(attention_mask) == 1,labels_ids,0))

[50259 50359 50363  9000   578 25866  9116  1246 14868  9436 18939  6244
 50257     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0]


In [32]:
# setting the parameters of model for correct working
model.generation_config.language = "english"
model.generation_config.task = "transcribe"
model.generation_config.forced_decoder_ids = None
model.freeze_encoder()

In [28]:
    
# load and prepare the dataset
dataset = load_from_disk(dataset_path)
ds_ready = dataset.map(prepare_dataset.prepare_dataset_with_prompt, remove_columns=dataset.column_names, num_proc=1)
# ds_ready = ds_ready.rename_columns({'labels_fullts':'labels','fullts_prompt_ids':'prompt_ids'})

print(ds_ready)

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

Dataset({
    features: ['input_features', 'attention_mask', 'long_callsigns', 'short_callsigns', 'labels', 'prompt_ids'],
    num_rows: 70
})


In [29]:
processor.tokenizer.decode(ds_ready[2]['labels'])

'<|startofprev|> air france one zero eight zulu<|startoftranscript|><|en|><|transcribe|><|notimestamps|>air france one zero eight zulu standby now descend to descend flight level seven zero air france one zero eight zulu<|endoftext|>'

In [17]:
inputs = processor(dataset[0]['audio']['array'], padding='max_length',  return_tensors="pt", truncation=False,  return_attention_mask=True, sampling_rate=16_000)
print(inputs.input_features.shape)

torch.Size([1, 80, 3000])


In [42]:
batch = ds_ready[0:4]

In [ ]:
init_tokens = processor.tokenizer.convert_tokens_to_ids(['<|startoftranscript|>','<|en|>','<|transcribe|>','<|notimestamps|>'])
prompt_features = [{'input_ids': batch['prompt_ids'][idx] + init_tokens} for idx,_ in enumerate(batch['prompt_ids'])]
prompt_batch = processor.tokenizer.ppadding="longest",ad(prompt_features, return_tensors="pt")

In [110]:
prompt_ids = (ds_ready[0:4]['prompt_ids'][0] + init_tokens)

In [44]:
# PREPARE THE BATCH FOR GENERATE
# we will create batch of four
batch = {
    'input_features': torch.tensor(batch['input_features']).cuda(),
    'decoder_input_ids': torch.tensor(prompt_batch['input_ids']).cuda(),
    'decoder_attention_mask': torch.tensor(np.where(prompt_batch['input_ids'] != 50257,1,0)).cuda(),
}


/tmp/ipykernel_150494/3792684042.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'decoder_input_ids': torch.tensor(prompt_batch['input_ids']).cuda(),


In [93]:
print(decoder_input_ids[0][0:18])
print(decoder_attention_mask[0][18])

tensor([50361, 21065, 10567,   665,  2446,  2665,  1100,  2475,    88,  6244,
          286, 19198,  1045,  1451,  1411, 50258, 50259, 50359],
       device='cuda:0')
tensor(1, device='cuda:0')


In [153]:
init_tokens = processor.tokenizer.convert_tokens_to_ids(['<|startoftranscript|>','<|en|>','<|transcribe|>','<|notimestamps|>'])

idx=2
padding = [50257 for _ in range(2)]
input_features = torch.tensor(ds_ready[idx]['input_features']).unsqueeze(0).cuda()
prompt_ids = np.array(ds_ready[idx]['prompt_ids'] + init_tokens + padding) # 50257 is the pad token id
prompt_prompt = torch.tensor(ds_ready[idx]['prompt_ids']).cuda()


decoder_input_ids = torch.tensor(prompt_ids).unsqueeze(0).cuda()
decoder_attention_mask = torch.tensor(np.where(prompt_ids != 50257,1,0)).unsqueeze(0).cuda()
decoder_position_ids = torch.arange(1, len(prompt_ids)+1).unsqueeze(0).cuda()
#for padding set decoder_position_ids to 0
decoder_position_ids[0][len(ds_ready[idx]['prompt_ids'] + init_tokens):] = 1

print(decoder_input_ids.shape, decoder_position_ids)
out2 = model.generate(
    input_features=input_features,
    decoder_input_ids=decoder_input_ids,
    decoder_attention_mask=decoder_attention_mask,
    decoder_position_ids=decoder_position_ids,
    # attention_mask=decoder_attention_mask
    # prompt_ids=torch.tensor(ds_ready[0]['prompt_ids']).cuda()
).detach().cpu()
print(processor.decode(out2[0],skip_special_tokens=True))

outx = model.generate(
    input_features=input_features,
    # decoder_input_ids=decoder_input_ids[1][0:18][None],
    # decoder_attention_mask=decoder_attention_mask,
    # attention_mask=decoder_attention_mask
    prompt_ids=prompt_prompt
).detach().cpu()
print(processor.decode(outx[0],skip_special_tokens=True))

torch.Size([1, 29]) tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
         19, 20, 21, 22, 23, 24, 25, 26, 27,  1,  1]], device='cuda:0')
 golf Yan
 Hotel Golf Yankee we appreciate the shortcut the Golf Yankee roger report your program in Bern


In [127]:
decoder_attention_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0]], device='cuda:0')

In [53]:
print(processor.decode(out2[0],skip_special_tokens=True))

 Hotel Golf Yankee we appreciate the shortcut the Golf Yankee roger report your program in Bern


In [11]:
init_tokens = processor.tokenizer.convert_tokens_to_ids(['<|startoftranscript|>','<|en|>','<|transcribe|>','<|notimestamps|>'])

In [24]:
# test modelu s promptem (1 nebo 10 epoch vyjde na stejno zda se)
dec_qwd=np.array(ds_ready[3]['prompt_ids']+init_tokens+ [50257 for _ in range(20)])
mask=np.where(dec_qwd != 50257,1,0)

out2 = model.generate(
    torch.tensor(ds_ready[3]['input_features']).unsqueeze(0).cuda(),
    decoder_input_ids=torch.tensor(dec_qwd).unsqueeze(0).cuda(),
    decoder_attention_mask=torch.tensor(mask).unsqueeze(0).cuda(),
    
    # prompt_ids=torch.tensor(ds_ready[3]['prompt_ids']).cuda()
).detach().cpu()
print(processor.decode(out2[0]))

out2 = model.generate(
    torch.tensor(ds_ready[3]['input_features']).unsqueeze(0).cuda(),
    # decoder_input_ids=torch.tensor(ds_ready[3]['prompt_ids']+init_tokens).unsqueeze(0).cuda()
    prompt_ids=torch.tensor(ds_ready[3]['prompt_ids']).cuda()
).detach().cpu()
print(processor.decode(out2[0]))

 vacate bravo one copy traffic three four left cleared to land Qantas Six Forty Two on a spot 93 vacate now via bravo one
 vacate bravo one copy traffic three four left cleared to land Qantas Six Forty Two on a spot 93 vacate now via bravo one


In [7]:
# test modelu, kdy je vstup a prompt jsou od jinych nahravek
# vysledek - neni to uplne tak, ze ten model prepsal ten prompt, ale dost se to blizi tomu
out3 = model.generate(
    torch.tensor(ds_ready[2]['input_features']).unsqueeze(0).cuda(),
    prompt_ids=torch.tensor(ds_ready[0]['prompt_ids']).cuda()
).detach().cpu()
print(processor.decode(out3[0]))
print('real')
print(processor.decode(ds_ready[1]['labels'],skip_special_tokens=True))

You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 Go to the taxi we appreciate the drop-up. The coffee and the rock show report you program in burn
real
ruwnay three four left cleared to land Jetstar Four Twenty One


In [28]:


# Prepare decoder prompts (pad them to the same length)
batch_decoder_inputs = processor.tokenizer.pad({"input_ids": ds_ready[0:2]['prompt_ids']}, padding=True, return_tensors="pt")

# Generate
outputs = model.generate(
    input_features=torch.tensor(ds_ready[0:1]['input_features']).cuda(),
    decoder_input_ids=batch_decoder_inputs["input_ids"].cuda(),
    decoder_attention_mask=batch_decoder_inputs["attention_mask"],  # Important for handling padding
)

IndexError: list assignment index out of range

In [49]:
# print(processor.decode(out1[0]))
print(processor.decode(ds_ready[0]['labels']))

<|startoftranscript|><|notimestamps|>Radar CSA One Delta Zulu established
CSA One Delta Zulu roger contact Ruzyne Tower one three four decimal five six zero

three four five six zero CSA One Delta Zulu  pekný deň<|endoftext|>


In [10]:
from typing import Any, Dict, List, Union
from dataclasses import dataclass
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors

        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        batch["labels"] = labels_batch
        
        return batch

In [11]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [23]:
dataloader = DataLoader(ds_ready, batch_size=3, collate_fn=data_collator)
metr = ComputeMetrics(processor.tokenizer)

In [ ]:
out = model.generate(input_features).detach().cpu()

In [ ]:

# Iterate over batches
all_preds = []
all_lables = []
for batch in dataloader:
    input_features = batch["input_features"].cuda()
    out = model.generate(input_features).detach().cpu()
    all_preds.extend(processor.batch_decode(out, skip_special_tokens=True))
    all_lables.extend(processor.batch_decode(batch["labels"]['input_ids'], skip_special_tokens=True))
    # Free memory
    del batch
    torch.cuda.empty_cache()


In [53]:
metr.compute_metrics(all_preds, all_lables)    

TypeError: 'int' object is not subscriptable

In [35]:
metr.compute_metrics2(all_preds, all_lables)    

{'wer': 76.61662817551964}

In [ ]:
val = metr.compute_metrics(out[0].unsqueeze(0), batch["labels"]['input_ids'][0].unsqueeze(0))
val1 = metr.compute_metrics(out[1].unsqueeze(0), batch["labels"]['input_ids'][1].unsqueeze(0))
val2= metr.compute_metrics(out[2].unsqueeze(0), batch["labels"]['input_ids'][2].unsqueeze(0))
print(val,val1,val2,(val1['wer']+val2['wer']+val['wer'])/3)

In [57]:
batch = torch.tensor(ds_ready['input_features'][:5]).cuda()

In [ ]:
out=model.generate(batch)

In [59]:
del batch
del model
torch.cuda.empty_cache()
torch.cuda.synchronize()

In [10]:
processor.tokenizer.decode(ds_ready[0]['labels'], skip_special_tokens=True)

'Radar CSA One Delta Zulu established\nCSA One Delta Zulu roger contact Ruzyne Tower one three four decimal five six zero\n\nthree four five six zero CSA One Delta Zulu  pekný deň'

In [11]:
out.shape

torch.Size([3, 42])

In [14]:
processor.tokenizer.decode(out[0], skip_special_tokens=True)

' Radar CC1DZ, established CC1DZ, roger, contact roger tower 134.560 134.560, CC1DZ, take me again'

In [31]:
metr = ComputeMetrics(processor.tokenizer)

In [16]:
out[0]

tensor([ 9654,   289, 12630,    16,    35,    57,    11,  7545, 12630,    16,
           35,    57,    11,   744,  1321,    11,  3385,   744,  1321, 10567,
         3705,    19,    13,    20,  4550,  3705,    19,    13,    20,  4550,
           11, 12630,    16,    35,    57,    11,   747,   385,   797, 50257,
        50257, 50257], device='cuda:0')

In [17]:
torch.tensor(ds_ready[0]['labels'])

tensor([50258, 50363, 48444,   289,   383,  8886,  1485, 18183,  1176, 12845,
         7545,   198,    34,  8886,  1485, 18183,  1176, 12845,   744,  1321,
         3385, 15702,  1229,   716, 17877,   472,  1045,  1451, 26601,  1732,
         2309,  4018,   198,   198, 27583,  1451,  1732,  2309,  4018,   383,
         8886,  1485, 18183,  1176, 12845,   220,   520,  5457, 11822,   368,
          129,   230, 50257])

In [18]:
metr.compute_metrics(out[0].unsqueeze(0), torch.tensor(ds_ready[0]['labels']).unsqueeze(0))

{'wer': 93.54838709677419}

In [44]:
out_test = processor.tokenizer.batch_decode(out, skip_special_tokens=True)
label_test = processor.tokenizer.batch_decode(ds_ready['labels'][:3], skip_special_tokens=True)

In [45]:
label_test.__len__()

3

In [46]:
metr.compute_metrics2(out_test,label_test)

{'wer': 78.37837837837837}